In [4]:
# --- Imports ---
import tensorflow as tf
import numpy as np
import time
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.optimizers import SGD, Adam, RMSprop
from tensorflow.keras.regularizers import l2
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

# --- Eager Execution ---
try:
    tf.config.run_functions_eagerly(True)
except Exception as e:
    print("Eager execution couldn't be set explicitly:", e)

# --- Load and Preprocess MNIST ---
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train / 255.0
x_test = x_test / 255.0
x_train = x_train.reshape(-1, 28, 28, 1)
x_test = x_test.reshape(-1, 28, 28, 1)
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

# --- Model Builder ---
def create_model(optimizer, regularizer_strength=0.01, use_dropout=False):
    model = Sequential()
    model.add(Flatten(input_shape=(28, 28, 1)))
    model.add(Dense(128, activation='relu', kernel_regularizer=l2(regularizer_strength)))

    if use_dropout:
        model.add(Dropout(0.5))

    model.add(Dense(64, activation='relu', kernel_regularizer=l2(regularizer_strength)))
    model.add(Dense(10, activation='softmax'))

    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# --- Train + Evaluate ---
def train_and_evaluate_model(optimizer, use_dropout=False):
    model = create_model(optimizer, use_dropout=use_dropout)
    start_time = time.time()

    history = model.fit(
        x_train, y_train,
        epochs=5,
        batch_size=32,
        validation_data=(x_test, y_test),
        verbose=2
    )

    elapsed_time = time.time() - start_time
    return history, elapsed_time

# --- Define Optimizer Classes ---
optimizer_classes = {
    "SGD": SGD,
    "Adam": Adam,
    "RMSprop": RMSprop
}

results = {}

# --- Run Experiments ---
for name, OptimizerClass in optimizer_classes.items():
    print(f"\nTraining with {name} optimizer (without dropout):")
    history, time_taken = train_and_evaluate_model(OptimizerClass(), use_dropout=False)
    results[f"{name}_no_dropout"] = {'history': history, 'time_taken': time_taken}

    print(f"\nTraining with {name} optimizer (with dropout):")
    history, time_taken = train_and_evaluate_model(OptimizerClass(), use_dropout=True)
    results[f"{name}_with_dropout"] = {'history': history, 'time_taken': time_taken}

# --- Output Final Results ---
for key, value in results.items():
    final_val_acc = value['history'].history['val_accuracy'][-1]
    print(f"\n{key}:")
    print(f"Final Validation Accuracy: {final_val_acc:.4f}")
    print(f"Time Taken: {value['time_taken']:.2f} seconds")



Training with SGD optimizer (without dropout):
Epoch 1/5
1875/1875 - 84s - 45ms/step - accuracy: 0.8286 - loss: 2.9371 - val_accuracy: 0.9040 - val_loss: 1.9749
Epoch 2/5
1875/1875 - 142s - 76ms/step - accuracy: 0.9071 - loss: 1.5488 - val_accuracy: 0.9167 - val_loss: 1.1876
Epoch 3/5
1875/1875 - 138s - 74ms/step - accuracy: 0.9161 - loss: 0.9938 - val_accuracy: 0.9181 - val_loss: 0.8174
Epoch 4/5
1875/1875 - 83s - 44ms/step - accuracy: 0.9212 - loss: 0.7249 - val_accuracy: 0.9258 - val_loss: 0.6312
Epoch 5/5
1875/1875 - 80s - 43ms/step - accuracy: 0.9250 - loss: 0.5891 - val_accuracy: 0.9304 - val_loss: 0.5352

Training with SGD optimizer (with dropout):
Epoch 1/5
1875/1875 - 95s - 51ms/step - accuracy: 0.7213 - loss: 3.1846 - val_accuracy: 0.8954 - val_loss: 2.0079
Epoch 2/5
1875/1875 - 90s - 48ms/step - accuracy: 0.8622 - loss: 1.6834 - val_accuracy: 0.9122 - val_loss: 1.2057
Epoch 3/5
1875/1875 - 145s - 77ms/step - accuracy: 0.8841 - loss: 1.0989 - val_accuracy: 0.9209 - val_loss: